In [27]:
!pip install pytesseract
!sudo apt install tesseract-ocr
!pip install Pillow
!pip install markdown

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [51]:
import requests
from PIL import Image
from io import BytesIO
import pytesseract
import json
import IPython
from markdown import markdown
# Fungsi untuk menampilkan output Markdown
Markdown = lambda string: IPython.display.HTML(markdown(string))
from google.colab import userdata

In [61]:


def download_image(image_url):
    """
    Download image from a given URL.

    Args:
        image_url (str): URL of the image.

    Returns:
        Image object if successful, None otherwise.
    """
    try:
        response = requests.get(image_url, stream=True)
        response.raise_for_status()  # Check for HTTP request errors
        return Image.open(BytesIO(response.content))
    except requests.exceptions.RequestException as e:
        print(f"Error downloading image: {e}")
        return None

def perform_ocr(image):
    """
    Perform OCR on the provided image.

    Args:
        image (Image): PIL Image object.

    Returns:
        str: Extracted text from the image.
    """
    try:
        return pytesseract.image_to_string(image).strip()
    except Exception as e:
        print(f"Error during OCR: {e}")
        return ""

def send_to_cloudflare_ai(model, image_url, prompt, bearer,user_id):
    """
    Send data to Cloudflare AI Worker.

    Args:
        model (str): AI model to use.
        image_url (str): URL of the image.
        prompt (str): Text prompt or message to accompany the image.

    Returns:
        dict: JSON response from the Cloudflare AI Worker.
    """
    url = f"https://api.cloudflare.com/client/v4/accounts/{user_id}/ai/run/{model}"
    headers = {
        "Authorization": f"Bearer {bearer}",
        "Content-Type": "application/json",
    }
    data = {
        "messages": [
            {
                "role": "system",
                "content": "kamu adalah assistent pribadi ku yang bisa melakukan coding standart"
            },
            {
                "role": "user",
                "content": prompt,
                "image": {
                    "url": image_url
                }
            }
        ]
    }
    try:
        response = requests.post(url, headers=headers, json=data)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error sending data to Cloudflare AI: {e}")
        return {"error": str(e)}

def simple_rag(model, image_url, prompt=None, bearer=None, user_id=None):
    """
    Perform OCR on an image and send data to Cloudflare AI Worker.

    Args:
        model (str): AI model to use.
        image_url (str): URL of the image to process.
        prompt (str): Optional text prompt.
        bearer (str): Require bearer Token for auth.
        user_id (str): Cloudflare Account ID.

    Returns:
        dict: JSON response from the AI worker or an error message.
    """
    image = download_image(image_url)
    if not image:
        return {"error": "Failed to download image."}

    extracted_text = perform_ocr(image)
    combined_prompt = prompt or ""
    if extracted_text:
        combined_prompt = f"{combined_prompt}\nExtracted text from image:\n{extracted_text}".strip()

    response = send_to_cloudflare_ai(model, image_url, combined_prompt, bearer, user_id)
    return response

def format_output(result, template):
    """
    Format the output based on the specified template.

    Args:
        result (dict): JSON response from the AI worker.
        template (str): The desired template format (raw, json, text, markdown).
    """
    response = result.get("result", {}).get("response", "")

    if template == "raw":
        print(response)

    elif template == "json":
        print(json.dumps({"response": response}, indent=4))

    elif template == "text":
        print(response)

    elif template == "markdown":
        return Markdown(response)

    else:
        print("Invalid template format specified.")

In [79]:
if __name__ == "__main__":
    model = "@cf/google/gemma-7b-it-lora"
    image_url = "https://i.ibb.co/Jt3Bb4V/image.png"
    prompt = "Analyze the content of this image and provide insights."
    bearer = userdata.get('bearer')
    user_id = userdata.get('user_id')

    result = simple_rag(model, image_url, prompt, bearer, user_id)

In [80]:
format_output(result,"raw")

## Analysis of Image Text Content

This image text describes two PHP errors encountered while executing code. It provides details for each error, including:

**Error 1:**

* **Error Message:** "A PHP Error was encountered. Seventy: Warning - Mossage:ini_set(): A session is active. You cannot change the session medule's in settings at this time"
* **Filename:** "Session/Session php"
* **Line Number:** 284
* **Bockeroce:** The text points to the specific file ('Session/Session php') and line number (284) where the error occurred.
* **Function:** Not specified in this text snippet.

**Error 2:**

* **Error Message:** "AHP Error was encountered. Severty: Warning - Message: session_set .cookie_porams(): Cannot change session cookie parameters when session is active"
* **Filename:** "Session/Session php"
* **Line Number:** 298
* **Bocktroce:** This error also points to the same file and line number as the first error.
* **Function:** Not specified in this text snippet.

**Insights:**

These 

In [81]:
format_output(result,"json")

{
    "response": "## Analysis of Image Text Content\n\nThis image text describes two PHP errors encountered while executing code. It provides details for each error, including:\n\n**Error 1:**\n\n* **Error Message:** \"A PHP Error was encountered. Seventy: Warning - Mossage:ini_set(): A session is active. You cannot change the session medule's in settings at this time\"\n* **Filename:** \"Session/Session php\"\n* **Line Number:** 284\n* **Bockeroce:** The text points to the specific file ('Session/Session php') and line number (284) where the error occurred.\n* **Function:** Not specified in this text snippet.\n\n**Error 2:**\n\n* **Error Message:** \"AHP Error was encountered. Severty: Warning - Message: session_set .cookie_porams(): Cannot change session cookie parameters when session is active\"\n* **Filename:** \"Session/Session php\"\n* **Line Number:** 298\n* **Bocktroce:** This error also points to the same file and line number as the first error.\n* **Function:** Not specified

In [82]:
format_output(result,"markdown")